# LangGraph with Claude API Demo

This notebook demonstrates a basic LangGraph application that interacts with the Claude API.

In [1]:
import os
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from anthropic import Anthropic
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Claude client
client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

In [2]:
# Define the state structure
class State(TypedDict):
    messages: Annotated[list, add_messages]
    user_input: str
    claude_response: str

In [7]:
# Which Claude model to use
model_id = "claude-sonnet-4-20250514"

# Define the Claude API call function
def call_claude(state: State) -> State:
    """Call Claude API with the user input"""
    user_message = state["user_input"]
    
    try:
        response = client.messages.create(
            model=model_id,
            max_tokens=1000,
            messages=[
                {"role": "user", "content": user_message}
            ]
        )
        
        claude_response = response.content[0].text
        
        return {
            "messages": [{"role": "assistant", "content": claude_response}],
            "claude_response": claude_response
        }
    except Exception as e:
        error_msg = f"Error calling Claude API: {str(e)}"
        return {
            "messages": [{"role": "assistant", "content": error_msg}],
            "claude_response": error_msg
        }

In [8]:
# Define a processing function
def process_input(state: State) -> State:
    """Process the user input before sending to Claude"""
    user_input = state["user_input"]
    
    # Add some basic preprocessing (you can expand this)
    processed_input = user_input.strip()
    
    return {
        "user_input": processed_input,
        "messages": [{"role": "user", "content": processed_input}]
    }

In [9]:
# Create the graph
workflow = StateGraph(State)

# Add nodes
workflow.add_node("process_input", process_input)
workflow.add_node("call_claude", call_claude)

# Define the flow
workflow.set_entry_point("process_input")
workflow.add_edge("process_input", "call_claude")
workflow.add_edge("call_claude", END)

# Compile the graph
app = workflow.compile()

In [10]:
# Test the application
def run_claude_chat(user_input: str):
    """Run the LangGraph application with user input"""
    initial_state = {
        "messages": [],
        "user_input": user_input,
        "claude_response": ""
    }
    
    result = app.invoke(initial_state)
    return result["claude_response"]

# Example usage
response = run_claude_chat("Hello! Can you tell me a fun fact about space?")
print("Claude's response:")
print(response)

Claude's response:
Here's a fascinating space fact: On Mars, sunsets are blue! 

While we're used to seeing orange and red sunsets on Earth due to our atmosphere scattering light, Mars has a much thinner atmosphere with fine dust particles that scatter light differently. This causes the sun to appear blue as it sets on the Red Planet - basically the opposite of what we see here on Earth!

NASA's rovers have actually captured images of these otherworldly blue sunsets, giving us a beautiful glimpse of what an evening would look like if you were standing on Mars. 🌅


In [ ]:
# Interactive chat function
def interactive_chat():
    """Simple interactive chat with Claude via LangGraph"""
    print("Chat with Claude! (type 'quit' to exit)")
    
    while True:
        user_input = input("\nYou: ")
        
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Goodbye!")
            break
        
        response = run_claude_chat(user_input)
        print(f"\nClaude: {response}")

# Uncomment the line below to start interactive chat
# interactive_chat()

## Setup Instructions

1. Create a `.env` file in your project directory with your Anthropic API key:
   ```
   ANTHROPIC_API_KEY=your_api_key_here
   ```

2. Run the cells in order to install dependencies and set up the LangGraph application.

3. Use `run_claude_chat("your message")` to send messages to Claude through the graph.

4. Uncomment the last line in the interactive chat cell to start a conversation.

## Graph Structure

The application follows this flow:
1. **process_input**: Preprocesses the user input
2. **call_claude**: Makes the API call to Claude
3. **END**: Returns the response

You can extend this by adding more nodes for additional processing, memory, tool usage, etc.